Environment: Harpy

### Import packages

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import anndata as ad
import sparrow as sp
import os
from napari_spatialdata import Interactive
import numpy as np
import pandas as pd
import geopandas as gpd
from skimage import io, morphology

from sparrow.utils._keys import _INSTANCE_KEY, _REGION_KEY
from spatialdata.models import ShapesModel

### Specify paths

In [ ]:
input_path = r'D:\Data\2024-11-NovaST-BrettBoonen-ThVo\data\processed'
output_path = os.path.join(input_path, 'harpy')
os.makedirs(output_path, exist_ok=True)

### Create sdata

In [ ]:
sdata = sp.io.create_sdata(
    input=os.path.join(input_path, r'NOS__9c1b29__Nose_Epithelum_10um_NovaST_HE_Aligned_inverted.tif'),
    output_path=os.path.join(output_path, "sdata.zarr"),
    img_layer="HE",
    chunks=1024,
    scale_factors=(2, 2, 2, 2),
)
sdata

### Add anndata

In [ ]:
def add_NovaST_adata(sdata, adata_name, adata_path, radius):
    # Read adata
    adata = ad.read_h5ad(adata_path)

    # Change indexes and add region and instance key
    adata.obs.reset_index(inplace=True)
    adata.obs.index = adata.obs.index + 1 
    adata.obs.index.name = 'cells'
    
    adata.obs[_REGION_KEY] = f"labels_spots_{adata_name}"
    adata.obs[_INSTANCE_KEY] = adata.obs.index

    # Add shapes and labels layer based on coordinates
    adata.obsm['spatial_pxl'] = adata.obsm['spatial']
    adata.obsm['spatial'] = adata.obsm['spatial_um']  
    
    labels_shape = (6930, 6967) # FIXME: is currently hardcoded
    labels_img = np.zeros(labels_shape, dtype=np.int32)

    for (x, y), cell_id in zip(adata.obsm['spatial'], adata.obs[_INSTANCE_KEY]):
        x, y = int(x), int(y)
        if 0 <= y < labels_shape[0] and 0 <= x < labels_shape[1]:
            labels_img[y, x] = cell_id
    
    sdata = sp.im.add_labels_layer(sdata, arr=labels_img, output_layer=f"labels_spots_{adata_name}", chunks=1024, scale_factors=(2, 2, 2, 2), overwrite=True)
    sdata = sp.im.expand_labels_layer(sdata, labels_layer=f"labels_spots_{adata_name}", distance=radius, output_labels_layer=f"labels_spots_{adata_name}", output_shapes_layer=f"spots_{adata_name}", chunks=1024, scale_factors=(2, 2, 2, 2), overwrite=True)

    # Add adata as table layer
    sdata = sp.tb.add_table_layer(sdata, adata=adata.copy(), output_layer=adata_name, region=[f"labels_spots_{adata_name}"], overwrite=True)

    return sdata


In [ ]:
adata_paths = {
    '10um': os.path.join(input_path, 'NOS__9c1b29__Nose_Epithelum_10um.h5ad'),
    '20um': os.path.join(input_path, 'NOS__9c1b29__Nose_Epithelum_20um.h5ad'),
    '40um': os.path.join(input_path, 'NOS__9c1b29__Nose_Epithelum_40um.h5ad'),
    '100um': os.path.join(input_path, 'NOS__9c1b29__Nose_Epithelum_100um.h5ad')
    }

for adata_name, adata_path in adata_paths.items():
    radius = int(adata_name[:-2]) / 2
    
    sdata = add_NovaST_adata(
        sdata,
        adata_name = adata_name, 
        adata_path = adata_path,
        radius = radius
    )

### Sum expression of genes of interest

In [ ]:
def sum_genes(sdata, adata_name, genes):
    adata = sdata.tables[adata_name].copy()

    sum = adata[:, genes].X.sum(axis=1)
    adata.obs['sum'] = np.array(sum).flatten()

    # Add adata as table layer
    sdata = sp.tb.add_table_layer(sdata, adata=adata.copy(), output_layer=adata_name, region=[f"labels_spots_{adata_name}"], overwrite=True)

    return sdata


In [ ]:
for adata_name in adata_paths:
    sdata = sum_genes(
        sdata, 
        adata_name=adata_name,
        genes = ['Trpm5', 'Pou2f3', 'Avil', 'Ltc4s']
    ) 

### Add tissue mask

In [ ]:
tissue_mask = io.imread(os.path.join(output_path, 'tissue_mask_ilastik.tiff'))
tissue_mask = (tissue_mask == 1)
tissue_mask = morphology.remove_small_objects(tissue_mask, min_size=50000)
tissue_mask = morphology.remove_small_holes(tissue_mask, area_threshold=5000)

sdata = sp.im.add_labels_layer(
    sdata, 
    arr=tissue_mask, 
    output_layer="tissue_mask", 
    chunks=1024, 
    overwrite=True)

### Filter out spots not in tissue

In [ ]:
def add_in_tissue(sdata, table_layer, tissue_mask):
    def assign_region(centroid, mask, mask_shape):
        x, y = int(round(centroid[0])), int(round(centroid[1]))
        
        if 0 <= y < mask_shape[0] and 0 <= x < mask_shape[1]:
            return 1 if mask[y, x] == 1 else 0
        return 0  # Outside mask bounds

    adata = sdata.tables[table_layer].copy()
    coordinates = adata.obsm['spatial']
    coordinates_df = pd.DataFrame(coordinates, columns=["x", "y"], index=adata.obs.index)

    mask_shape = tissue_mask.shape
    adata.obs["in_tissue"] = coordinates_df.apply(
        lambda row: assign_region((row["x"], row["y"]), tissue_mask, mask_shape), axis=1
    )

    sdata = sp.tb.add_table_layer(
        sdata,
        adata=adata.copy(),
        output_layer=table_layer,
        region=[f"labels_spots_{table_layer}"],
        overwrite=True,
    )
    
    adata_in_tissue = adata[adata.obs["in_tissue"] == 1].copy()
    
    sdata = sp.tb.add_table_layer(
        sdata,
        adata=adata_in_tissue.copy(),
        output_layer=f'{table_layer}_in_tissue',
        region=[f"labels_spots_{table_layer}"],
        overwrite=True,
    )

    return sdata


In [ ]:
for adata_name in adata_paths:
    sdata = add_in_tissue(
        sdata, 
        table_layer = adata_name, 
        tissue_mask = sdata.labels['tissue_mask'].compute().data
    )

### Plots

In [ ]:
for adata_name in adata_paths:
    output_path_plots = os.path.join(output_path, adata_name)
    os.makedirs(output_path_plots, exist_ok=True)
    
    crd = [4000,5000, 2500,3500]
    
    sp.pl.plot_shapes(
        sdata,
        img_layer = "HE",
        channel = 0,
        figsize = (25,25),
        output = os.path.join(output_path_plots, "HE")
    )
    
    sp.pl.plot_shapes(
        sdata,
        img_layer = "HE",
        channel = 0,
        crd = crd,
        figsize = (25,25),
        output = os.path.join(output_path_plots, "crop_HE")
    )
    
    for column in ['total_counts', 'n_genes_by_counts', 'Trpm5', 'Pou2f3', 'Avil', 'Ltc4s', 'sum']:        
        sp.pl.plot_shapes(
            sdata,
            img_layer = "HE",
            shapes_layer = f"spots_{adata_name}",
            channel = 0,
            table_layer = adata_name,
            column = column,
            alpha = 1,
            linewidth = 0,
            figsize = (25,25),
            output = os.path.join(output_path_plots, column)
        )
        
        sp.pl.plot_shapes(
            sdata,
            img_layer = "HE",
            shapes_layer = f"spots_{adata_name}",
            channel = 0,
            crd = crd,
            table_layer = adata_name,
            column = column,
            alpha = 0.7,
            linewidth = 0,
            figsize = (25,25),
            output = os.path.join(output_path_plots, f'crop_{column}')
        )
        
        sp.pl.plot_shapes(
            sdata,
            img_layer = "HE",
            shapes_layer = f"spots_{adata_name}",
            channel = 0,
            table_layer = f'{adata_name}_in_tissue',
            column = column,
            alpha = 1,
            linewidth = 0,
            figsize = (25,25),
            output = os.path.join(output_path_plots, f'tissue_{column}')
        )
        
        sp.pl.plot_shapes(
            sdata,
            img_layer = "HE",
            shapes_layer = f"spots_{adata_name}",
            channel = 0,
            crd = crd,
            table_layer = f'{adata_name}_in_tissue',
            column = column,
            alpha = 0.7,
            linewidth = 0,
            figsize = (25,25),
            output = os.path.join(output_path_plots, f'tissue_crop_{column}')
        )

### Leiden clustering

In [ ]:
import gc # garbage collection to avoid out-of-memory issues.

for adata_name in adata_paths:
    print(f'running: {adata_name}')
    
    output_path_plots = os.path.join(output_path, adata_name)
    os.makedirs(output_path_plots, exist_ok=True)
    
    crd = [4000,5000, 2500,3500]
    
    # Preprocess data
    sdata = sp.tb.preprocess_transcriptomics(
        sdata=sdata,
        size_norm=False,
        labels_layer=f"labels_spots_{adata_name}",
        table_layer=f'{adata_name}_in_tissue',
        output_layer=f"{adata_name}_in_tissue_processed",
        min_counts=50,
        min_cells=5,
        highly_variable_genes=True,
        max_value_scale=10,
        n_comps = 50,
        update_shapes_layers=False,
        overwrite=True
    )
    
    gc.collect()
    
    # Leiden clustering
    leiden_resolutions = [0.4, 0.6, 0.8]

    for res in leiden_resolutions:
        res_str = str(res).replace('.', '')
        
        sdata = sp.tb.leiden(
            sdata, 
            labels_layer=f"labels_spots_{adata_name}",
            table_layer=f"{adata_name}_in_tissue_processed",
            output_layer=f"{adata_name}_in_tissue_processed",
            calculate_umap=True,
            calculate_neighbors=True,
            rank_genes=True,
            n_neighbors=35,
            n_pcs=30,
            resolution=res,
            key_added=f'leiden_{res_str}',
            index_names_var=None,
            index_positions_var=None,
            random_state=100,
            overwrite=True
        )
        
        sp.pl.cluster(
            sdata, 
            table_layer = f"{adata_name}_in_tissue_processed",
            key_added=f'leiden_{res_str}',
            output=os.path.join(output_path_plots, f'UMAP_leiden_{res_str}')
        )
        
        sp.pl.plot_shapes(
            sdata,
            img_layer = "HE",
            shapes_layer = f"spots_{adata_name}",
            channel = 0,
            table_layer = f"{adata_name}_in_tissue_processed",
            column = f'leiden_{res_str}',
            alpha = 1,
            linewidth = 0,
            cmap = "rainbow",
            figsize = (25,25),
            output = os.path.join(output_path_plots, f'leiden_{res_str}')
        )
        
        sp.pl.plot_shapes(
            sdata,
            img_layer = "HE",
            shapes_layer = f"spots_{adata_name}",
            channel = 0,
            crd = crd,
            table_layer = f"{adata_name}_in_tissue_processed",
            column = f'leiden_{res_str}',
            alpha = 0.8,
            linewidth = 0,
            cmap = "rainbow",
            figsize = (25,25),
            output = os.path.join(output_path_plots, f'crop_leiden_{res_str}')
        )
    
        gc.collect()

### Save adata tables

In [ ]:
for adata_name in adata_paths:
    
    output_path_plots = os.path.join(output_path, adata_name)
    os.makedirs(output_path_plots, exist_ok=True)

    sdata.tables[adata_name].write(f'{output_path_plots}/adata_{adata_name}.h5ad')
    sdata.tables[f'{adata_name}_in_tissue_processed'].write(f'{output_path_plots}/adata_{adata_name}_in_tissue_processed.h5ad')

## OPTIONAL: Explore sdata interactively

In [ ]:
Interactive(sdata)